# Estimate Binomial draw probabilility

In [1]:
using SRDynamicHMC

Define a structure to hold the data.

In [2]:
struct BernoulliProblem
    "Total number of draws in the data."
    n::Int
    "Number of draws `==1` in the data"
    s::Vector{Int}
end;

Make the type callable with the parameters *as a single argument*.

In [3]:
function (problem::BernoulliProblem)((α, )::NamedTuple{(:α, )})
    @unpack n, s = problem        # extract the data
    loglikelihood(Binomial(n, α), s)
end

Create the data and complete setting up the problem.

In [4]:
obs = rand(Binomial(9, 2/3), 1)
p = BernoulliProblem(9, obs)
p((α = 0.5, ))

-4.041100047703289

Write a function to return properly dimensioned transformation.

In [5]:
problem_transformation(p::BernoulliProblem) =
    as((α = as𝕀, ),  )
# Use a flat priors (the default, omitted) for α
P = TransformedLogDensity(problem_transformation(p), p)
∇P = LogDensityRejectErrors(ADgradient(:ForwardDiff, P));

Sample

In [6]:
chain, NUTS_tuned = NUTS_init_tune_mcmc(∇P, 1000)

MCMC, adapting ϵ (75 steps)
0.0013 s/step ...done
MCMC, adapting ϵ (25 steps)
1.1e-5 s/step ...done
MCMC, adapting ϵ (50 steps)
0.0025 s/step ...done
MCMC, adapting ϵ (100 steps)
8.9e-6 s/step ...done
MCMC, adapting ϵ (200 steps)
8.0e-6 s/step ...done
MCMC, adapting ϵ (400 steps)
7.9e-6 s/step ...done
MCMC, adapting ϵ (50 steps)
1.0e-5 s/step ...done
MCMC (1000 steps)
6.7e-6 s/step ...done


(NUTS_Transition{Array{Float64,1},Float64}[NUTS_Transition{Array{Float64,1},Float64}([2.86732], -4.92578, 1, DoubledTurn, 1.0, 1), NUTS_Transition{Array{Float64,1},Float64}([3.16645], -4.62687, 2, DoubledTurn, 0.987731, 3), NUTS_Transition{Array{Float64,1},Float64}([2.93305], -4.62349, 1, DoubledTurn, 1.0, 1), NUTS_Transition{Array{Float64,1},Float64}([1.31305], -3.94207, 2, DoubledTurn, 1.0, 3), NUTS_Transition{Array{Float64,1},Float64}([0.557012], -3.90411, 1, AdjacentTurn, 0.809965, 3), NUTS_Transition{Array{Float64,1},Float64}([0.411088], -4.53864, 1, DoubledTurn, 0.869907, 1), NUTS_Transition{Array{Float64,1},Float64}([0.976492], -3.88075, 1, DoubledTurn, 1.0, 1), NUTS_Transition{Array{Float64,1},Float64}([0.976492], -4.63824, 1, DoubledTurn, 0.384041, 1), NUTS_Transition{Array{Float64,1},Float64}([1.86239], -3.49184, 1, AdjacentTurn, 0.955795, 3), NUTS_Transition{Array{Float64,1},Float64}([1.85006], -3.13387, 1, DoubledTurn, 1.0, 1)  …  NUTS_Transition{Array{Float64,1},Float64}([

To get the posterior for ``α`` use `get_position` and then transform back.

In [7]:
posterior = TransformVariables.transform.(Ref(problem_transformation(p)), get_position.(chain));

Extract the parameter.

In [8]:
posterior_α = first.(posterior);

check the effective sample size

In [9]:
ess_α = effective_sample_size(posterior_α)

433.7125631531574

NUTS-specific statistics

In [10]:
NUTS_statistics(chain)

Hamiltonian Monte Carlo sample of length 1000
  acceptance rate mean: 0.91, min/25%/median/75%/max: 0.03 0.89 0.97 1.0 1.0
  termination: AdjacentTurn => 35% DoubledTurn => 65%
  depth: 1 => 66% 2 => 34% 3 => 0%


check the mean

In [11]:
mean(posterior_α)

0.8189866261221316

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*